In [ ]:
########################################################################################################################
## -- libraries and packages -- ########################################################################################
########################################################################################################################
import os
import sys
sys.path.append(os.path.abspath(".."))
import torch
import transformer

########################################################################################################################
## -- testing the data handler module -- ###############################################################################
########################################################################################################################
src_vocab_path = "../data/vocabs/en_vocab.json"
tgt_vocab_path = "../data/vocabs/fa_vocab.json"
src_path, src_name = "../data/dataset/Tatoeba.zip", "en.txt"
tgt_path, tgt_name = "../data/dataset/Tatoeba.zip", "fa.txt"
SOS_TOKEN, PAD_TOKEN, EOS_TOKEN = '<SOS>', '<PAD>', '<EOS>'

data_handler = transformer.DataHandler(src_path, src_name, src_vocab_path, tgt_path, tgt_name, tgt_vocab_path, 
                                       SOS_TOKEN, PAD_TOKEN, EOS_TOKEN, max_sequence_length = 256, max_sentences = 1000)

data = data_handler.data()

########################################################################################################################
## -- testing the transformer encoder -- ###############################################################################
########################################################################################################################
batch_size, max_sequence_length, model_emb, num_heads, hidden, dropout_p, num_layers = 32, 256, 512, 8, 2048, 0.1, 4

encoder = transformer.TransformerEncoder(model_emb, num_heads, hidden, dropout_p, num_layers, data.src_stoi,
                                         max_sequence_length, SOS_TOKEN, PAD_TOKEN, EOS_TOKEN, device = 'cpu')

en_batch, fa_batch = ("Hi",), ("سلام",)
mask, _, _ = transformer.MaskGenerator(max_sequence_length = max_sequence_length).generate_masks(en_batch, fa_batch)
encoder(en_batch, enc_sos_token = True, enc_eof_token = True, mask = mask).shape

########################################################################################################################
## -- testing the transformer encoder -- ###############################################################################
########################################################################################################################
batch_size, max_sequence_length, model_emb, num_heads, hidden, dropout_p, num_layers = 32, 256, 512, 8, 2048, 0.1, 4
decoder = transformer.TransformerDecoder(model_emb, num_heads, hidden, dropout_p, num_layers, data.tgt_stoi,
                                         max_sequence_length, SOS_TOKEN, PAD_TOKEN, EOS_TOKEN, device = 'cpu')
x = torch.randn(batch_size, max_sequence_length, model_emb)
en_batch, fa_batch = ("Hi",), ("سلام",)
_, mask, cross_mask = transformer.MaskGenerator(max_sequence_length = max_sequence_length).generate_masks(en_batch, fa_batch)
decoder(x, fa_batch, dec_sos_token = True, dec_eos_token = True, mask = mask, cross_mask = cross_mask).shape

torch.Size([1, 256, 512])